### 导入gpd包

In [ ]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

### gpd的核心数据结构是geodataframe和geoseries，它们是pandas.dataframe和series的子类。

### gdf和gse的不同点在于，增加了***几何列：geometry***，和***坐标系：crs***


### GeoSeries 是存储单个几何类型（如全部是点，或全部是面）的一维数组，每个元素是 Shapely 几何对象（Point、LineString、Polygon 等）。

#### 1.通过sharely几何对象，Point创建

In [ ]:
points = [
    Point(116.3, 39.9),  # 北京（经纬度）
    Point(121.4, 31.2),  # 上海
    Point(113.2, 23.1)   # 广州
]
#指定crs
crs = 'EPSG:4362'
geo_points = gpd.GeoSeries(points,crs=crs)
display(geo_points)

### GeoDataFrame 是二维表格数据，包含多个属性列（如名称、人口等）和一个特殊的 “geometry” 列（存储几何对象），所有几何对象必须属于同一坐标系。

### 1.先创建常规pandas.df，再指定geometry，创建gdf`

In [ ]:
# 从坐标数据创建GeoDataFrame
data = {
"City": ["Tokyo", "New York", "London", "Paris"],
"Latitude": [35.6895, 40.7128, 51.5074, 48.8566],
"Longitude": [139.6917, -74.0060, -0.1278, 2.3522],
}
df = pd.DataFrame(data)
#指定几何列
gdf = gpd.GeoDataFrame(df,geometry=gpd.points_from_xy(df.Longitude, df.Latitude))
display(gdf)

gpd.points_from_xy() 函数是一个便利方法，从单独的经度（x）和纬度（y）列创建 Point 几何。
注意经度在前（x 坐标），然后是纬度（y 坐标）——这遵循标准数学约定

### 2.直接创建包含geometry的gdf

In [ ]:
# 从坐标数据创建GeoDataFrame
data = {
"City": ["Tokyo", "New York", "London", "Paris"],
"Latitude": [35.6895, 40.7128, 51.5074, 48.8566],
"Longitude": [139.6917, -74.0060, -0.1278, 2.3522],
'geometry':[
    Point(139.6917, 35.6895),   # Tokyo 
    Point(-74.0060, 40.7128),  # New York
    Point(-0.1278, 51.5074),   # London 
    Point(2.3522, 48.8566)     # Paris
]
}
"""或者
        data = {
    "City": ["Tokyo", "New York", "London", "Paris"],
    "Latitude": [35.6895, 40.7128, 51.5074, 48.8566],
    "Longitude": [139.6917, -74.0060, -0.1278, 2.3522],
    'geometry':gpd.points_from_xy([139.6917, -74.0060, -0.1278, 2.3522], [35.6895, 40.7128, 51.5074, 48.8566])
    }
"""
gdf = gpd.GeoDataFrame(data,geometry='geometry')
display(gdf)

需要导入Point类，from shapely.geometry import Point

### GDF关键属性和方法
属性\ 方法	|说明
|----|----|
gdf.geometry	|返回几何列（GeoSeries）
gpd.geometry.x/y|读取经度/纬度，series
gdf.crs|	指定或者返回坐标系信息（如 EPSG:4326）
gdf.to_crs|转换为其他坐标系，返回新数据或在元数据上修改
gdf.bounds|	返回每个几何的外接矩形（minx, miny, maxx, maxy）
gdf.total_bounds|	返回整个数据集的外接矩形
gdf.shape|	返回数据维度（行数，列数）
gdf.set_index()|设置索引，可选择columns

In [ ]:
#支持直接修改crs
print("CRS:", gdf.crs)
gdf.crs = 'EPSG:4326'  # WGS84
print("CRS:", gdf.crs)
#转换坐标系
gdf_new = gdf.to_crs('EPSG:3857')  # Web Merc
print("New CRS:", gdf_new.crs)
display(gdf_new)
#返回total外接矩形
bounds = gdf.total_bounds
print("Total Bounds:", bounds)  # (minx, miny, maxx, maxy)
#几何列
geometry = gdf.geometry
print("Geometry Column:\n", geometry)
metry_x = gdf.geometry.x
print("Geometry X:\n", metry_x)
#数据维度
shape = gdf.shape
print("Shape:", shape)  # (rows, columns)
#设置索引
gdf = gdf.set_index('City')
display(gdf)

## 文件读取
### gpd.read_file()函数自动检测文件格式和坐标系统，加载几何形状及其关联属性
可以读取.geojson,.shp文件等，***csv文件需要用pd读取***

In [ ]:
# 1. 读取 Shapefile（.shp）
gdf = gpd.read_file("path/to/shapefile.shp")

# 2. 读取 GeoJSON（.geojson）
gdf = gpd.read_file("path/to/data.geojson")

### csv读取
1.df = pd.read_csv()  
2.gpd.GeoDataFrame(df,...)

In [ ]:

# 3. 读取 CSV（需指定经纬度列转换为几何）
import pandas as pd
df = pd.read_csv("path/to/cities.csv")  # 包含 lon, lat 列
gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df["lon"], df["lat"]),
    crs="EPSG:4326"
)

### 文件写入
对于geojson和shapefile
### gpd.to_file(path_name,***driver="GeoJSON"***)

In [ ]:
#1. 写入 Shapefile
gdf.to_file("output/cities.shp")

# 2. 写入 GeoJSON
gdf.to_file("output/cities.geojson", driver="GeoJSON")

GeoPandas GeoDataFrame 本身没有一个直接将几何对象（如 Point, Polygon 等）以其原生格式写入普通 CSV 文件的内置方法，因为 CSV 文件是纯文本的表格数据，无法直接存储复杂的几何对象。

当你尝试直接使用 gdf.to_csv() 时，geometry 列中的 Shapely 对象会被转换为它们的字符串表示（例如 <shapely.geometry.point.Point object at 0x...>），这对于后续的地理空间操作是无用的。

因此，将 GeoPandas GeoDataFrame 写入 CSV 的核心思想是：将几何对象转换为文本表示形式（最常见的是 WKT 或 Well-Known Text），然后再写入 CSV。

### 方法1

### 对于只包含Point对象的gdf，使用gdf.geometry.x/y提取x和y
写入csv，   
创建一个不包含原始 'geometry' 列的 DataFrame   
也可以选择不删除，但 'geometry' 列在 CSV 中会是无用的对象字符串  
可以选择从geometry提取x,y，然后删除geometry几何列

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# 1. 创建一个示例 GeoDataFrame (只包含点)
data = {
    'id': [1, 2, 3, 4],
    'name': ['Store A', 'Park B', 'Office C', 'Home D'],
    'latitude_orig': [34.0522, 34.0522, 39.9042, 40.7128], # 原始纬度列
    'longitude_orig': [-118.2437, -118.2437, 116.4074, -74.0060], # 原始经度列
    'value': [100, 150, 200, 250]
}
df_temp = pd.DataFrame(data)
#建立gdf，设置几何列
geometry = [Point(xy) for xy in zip(df_temp['longitude_orig'], df_temp['latitude_orig'])]
gdf_points = gpd.GeoDataFrame(df_temp, geometry=geometry, crs="EPSG:4326")

display(gdf_points.head())

# 2. 从几何对象中提取经纬度
gdf_points['longitude'] = gdf_points.geometry.x
gdf_points['latitude'] = gdf_points.geometry.y

# 3. 创建一个不包含原始 'geometry' 列的 DataFrame
#移除geometry列
df_to_save_latlon = gdf_points.drop(columns=['geometry', 'latitude_orig', 'longitude_orig']) # 也可以保留原始经纬度列
display(df_to_save_latlon.head())

# 4. 写入 CSV 文件
output_csv_path_latlon = '../data/locations_with_latlon.csv'
df_to_save_latlon.to_csv(output_csv_path_latlon, index=False)


# -------------------------------------------------------------------
# 如何从包含经纬度列的 CSV 重新读取为 GeoDataFrame
# -------------------------------------------------------------------
print(f"\n尝试从 '{output_csv_path_latlon}' 重新读取...")
df_reloaded_latlon = pd.read_csv(output_csv_path_latlon)

# 将经纬度列转换回 Shapely 几何对象
geometry_reloaded_latlon = [Point(xy) for xy in zip(df_reloaded_latlon['longitude'], df_reloaded_latlon['latitude'])]

# 重新创建 GeoDataFrame
gdf_reloaded_latlon = gpd.GeoDataFrame(df_reloaded_latlon,
                                       geometry=geometry_reloaded_latlon,
                                       crs="EPSG:4326")

print("\n重新读取后的 GeoDataFrame (仅点):")
print(gdf_reloaded_latlon.head())
print("\n重新读取后的 GeoDataFrame 数据类型:")
print(gdf_reloaded_latlon.dtypes)
print("\n重新读取后的 GeoDataFrame CRS:")
print(gdf_reloaded_latlon.crs)

## 2.对于包含其他类型的gdf，如点、线、面等，将几何对象转换为 WKT (Well-Known Text) 字符串

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString

# 1. 创建一个示例 GeoDataFrame
# 假设我们有之前读取的 GeoDataFrame
data = {
    'id': [1, 2, 3, 4],
    'name': ['Store A', 'Park B', 'Office C', 'Home D'],
    'latitude': [34.0522, 34.0522, 39.9042, 40.7128],
    'longitude': [-118.2437, -118.2437, 116.4074, -74.0060],
    'value': [100, 150, 200, 250]
}
df_temp = pd.DataFrame(data)
geometry = [Point(xy) for xy in zip(df_temp['longitude'], df_temp['latitude'])]
gdf = gpd.GeoDataFrame(df_temp, geometry=geometry, crs="EPSG:4326")

# 添加一个 LineString 示例来展示 WKT 的通用性
line_geom = LineString([(0, 0), (1, 1), (2, 0)])
gdf.loc[len(gdf)] = {'id': 5, 'name': 'Path E', 'latitude': None, 'longitude': None, 'value': 300, 'geometry': line_geom}


print("原始 GeoDataFrame:")
print(gdf)
print("\nGeoDataFrame 数据类型:")
print(gdf.dtypes)

# 2. 将几何对象转换为 WKT 字符串
# 使用 .wkt 属性或 shapely.wkt.dumps()
gdf['geometry_wkt'] = gdf['geometry'].apply(lambda geom: geom.wkt if geom else None)

# 3. 创建一个不包含原始 'geometry' 列的 DataFrame
# 也可以选择不删除，但 'geometry' 列在 CSV 中会是无用的对象字符串
df_to_save = gdf.drop(columns=['geometry'])

# 4. 写入 CSV 文件
output_csv_path_wkt = '../data/locations_with_wkt.csv'
df_to_save.to_csv(output_csv_path_wkt, index=False) # index=False 避免写入 DataFrame 索引作为一列

print(f"\n数据已成功写入到 '{output_csv_path_wkt}'")
print("\n写入 CSV 后的 DataFrame 预览:")
print(df_to_save.head())

# -------------------------------------------------------------------
# 如何从包含 WKT 的 CSV 重新读取为 GeoDataFrame
# -------------------------------------------------------------------
from shapely import wkt

print(f"\n尝试从 '{output_csv_path_wkt}' 重新读取...")
df_reloaded = pd.read_csv(output_csv_path_wkt)

# 将 WKT 字符串转换回 Shapely 几何对象
# 注意：需要处理可能为 NaN 的情况，因为 LineString 示例的 lat/lon 为 None
geometry_reloaded = [wkt.loads(s) if pd.notna(s) else None for s in df_reloaded['geometry_wkt']]

# 重新创建 GeoDataFrame
gdf_reloaded = gpd.GeoDataFrame(df_reloaded.drop(columns=['geometry_wkt']),
                                geometry=geometry_reloaded,
                                crs="EPSG:4326") # 重新指定 CRS

print("\n重新读取后的 GeoDataFrame:")
print(gdf_reloaded.head())
print("\n重新读取后的 GeoDataFrame 数据类型:")
print(gdf_reloaded.dtypes)
print("\n重新读取后的 GeoDataFrame CRS:")
print(gdf_reloaded.crs)


`from shapely import wkt `  
提供了用于在几何对象和 Well-Known Text (WKT) 字符串之间进行转换的函数。  
wkt.loads(wkt_string)：将 WKT 字符串加载（load）为 Shapely 几何对象。  wkt.dumps(geometry_object)：将 Shapely 几何对象转储（dump）为 WKT 字符串。  
`gdf['geometry_wkt'] = gdf['geometry'].apply(lambda geom: geom.wkt if geom else None)`  
geom.wkt 它将 GeoDataFrame 中存储的复杂几何对象（Shapely 对象）转换为标准的、可读的文本格式。  
***也可以使用`gdf['geometry_wkt'] = gdf['geometry'].apply(lambda geom: wkt.dumps(geom) if geom else None)`***   
`geometry_reloaded = [wkt.loads(s) if pd.notna(s) else None for s in df_reloaded['geometry_wkt']]`  
pd.notna()判断是否为空值  
wkt.loads(s)接收一个 WKT 字符串作为输入，并将其解析成对应的 shapely 几何对象。如，wkt.loads("POINT (10 20)") 会返回 Point(10, 20) 对象。

## 坐标系和重投影


### 坐标系类型
地理坐标系（Geographic CRS）：用经纬度（度）表示位置，如 EPSG:4326（WGS84，全球通用）；  
投影坐标系（Projected CRS）：将球面坐标转换为平面坐标（米为单位），如 EPSG:3857（Web 墨卡托，适合 Web 地图）、EPSG:32650（UTM 分区 50N，适合局部区域精确计算）。

### 0.查看坐标系
使用gdf.crs查看原始坐标系  
gdf.crs.to_wkt()  # 输出Well-Known Text格式的CRS定义

### 1.坐标系指定
使用gdf.crs():设置或覆盖 坐标系,***但不执行任何坐标转换，不会改变坐标值***   
和gdf.set_crs(crs: 要设置的 CRS。  
allow_override=False (默认): 如果 GeoDataFrame 已经有 CRS，会抛出错误，防止意外覆盖。  
allow_override=True: 即使 GeoDataFrame 已经有 CRS，也会强制覆盖。)指定坐标系，***但不执行任何坐标转换，不会改变坐标值***

### 2.坐标系转换
gdf.to_crs()   
功能： 转换 GeoDataFrame 的 CRS。它会根据当前的 CRS 和目标 CRS，重新计算所有几何对象的坐标值。  
*前提*： GeoDataFrame 必须已经有一个正确的 CRS，否则 to_crs() 将不知道如何进行转换，会报错。  
***返回一个新的 GeoDataFrame，其中所有几何对象的坐标都已根据目标 CRS 进行了转换。原始 GeoDataFrame 不受影响***

# 几何创建

### 几何对象（Geometries）是由底层库 Shapely 创建和表示的。GeoPandas 的 GeoDataFrame 中的 geometry 列存储的就是 Shapely 几何对象。

### 几何对象类型包括：
1. Point
2. LineString
3. Polygon
4. MultiPoint,MulitLineString,MultiPolygon

### 1. Point
点是最基础的几何类型，由 x、y 坐标（2D）或 x、y、z 坐标（3D）定义。  
创建方法：  
通过Point()函数传入坐标值（单个数值或元组）。  
`Point(x, y, z)`

### 2. 线（LineString） 
线由有序的点集合组成，代表一条连续的线段。点的顺序决定了线的走向。  
创建方法：  
通过LineString()函数传入点的列表（每个点为元组或 Point 对象）。   
` LineString([(0, 0), (1, 1), (2, 0)]) `  
*注意：  
至少需要 2 个点才能创建 LineString，否则会报错。  
线的长度可通过.length属性直接获取。*


### 3. 多边形（Polygon）
多边形由外环（exterior） 和可选的内环（interior，即孔洞） 组成，代表一个封闭的区域。  
创建方法：  
通过Polygon()函数传入外环坐标列表（需闭合），可选传入内环列表（每个内环也是闭合的坐标列表）。  
`Polygon(outer_ring=[outerr_ring], holes=[inner_ring])`

### 4. 几何集合 (Multi-Geometries)
用于表示同类型但分离的多个几何对象。

`MultiPoint (多点): MultiPoint([point1, point2, ...]) `或` MultiPoint([(x1, y1), (x2, y2), ...])  `  
`MultiLineString (多线): MultiLineString([line1, line2, ...])  `  
`MultiPolygon (多面): MultiPolygon([polygon1, polygon2, ...])  `  


## Geopandas：管理几何对象的 GeoDataFrame
geopandas 扩展了 pandas 的 DataFrame，增加了一个特殊的geometry列，用于存储 shapely 创建的几何对象，同时支持坐标参考系统（CRS）和空间操作。

In [ ]:
from shapely.geometry import Point, LineString, Polygon, MultiPolygon

# 1. 创建点（Point）
point = Point(90, 20)  # （x=经度，y=纬度）

# 2. 创建线（LineString）：由点列表组成
line = LineString([(100, 20), (110,20), (130, 70)])

# 3. 创建面（Polygon）：由外环点列表组成（需闭合，首尾点一致）
polygon = Polygon([
    (116.3, 39.9), (116.4, 39.9), (116.4, 40.0), (116.3, 40.0), (116.3, 39.9)
])

# 4. 创建多面（MultiPolygon）：由多个Polygon组成
poly1 = Polygon([ (110, 40), (120, 40), (120, 50), (110,50)])
poly2 = Polygon([ (130, 30), (140, 30), (140, 40), (130,40)])
multi_poly = MultiPolygon([poly1, poly2])

# 5. 将几何对象添加到GeoDataFrame
gdf = gpd.GeoDataFrame(
    {"type": ["点", "线", "面", "多面"]},
    geometry=[point, line, polygon, multi_poly],
    crs="EPSG:4326"
)
display(gdf)
gdf.plot(column='type', legend=True, figsize=(10, 6))

# 几何属性和操作

一、几何属性 (Attributes)
几何属性通常是几何对象固有的特征，直接通过点号 (.) 访问。

1. 通用属性 (适用于所有几何类型)  
geom_type: 几何类型（如 'Point', 'LineString', 'Polygon', 'MultiPoint' 等）。  
is_empty: 如果几何对象不包含任何点，则为 True。  
is_simple: 如果几何对象没有自交或不必要的顶点，则为 True (对线和多边形有意义)。     
has_z: 如果几何对象包含 Z 坐标（三维），则为 True。  
bounds: 返回几何对象的 (minx, miny, maxx, maxy) 边界框。    
area: 几何对象的面积（对于点和线为 0.0）。    
length: 几何对象的长度（对于点和多边形为 0.0）。    
boundary: 几何对象的边界（例如，多边形的边界是线）。  
centroid: 几何对象的质心（Point 对象）。  
envelope:几何的外接矩形 （Polygon 对象）。  
convex_hull: 包含几何对象所有点的最小凸多边形。  
1. 特定几何类型属性  
Point:  
x: 点的 x 坐标。  
y: 点的 y 坐标。  
z: 点的 z 坐标（如果存在）。  
coords: 返回一个迭代器，包含点的坐标元组。  
LineString:  
coords: 返回一个迭代器，包含线的所有坐标元组。  
Polygon:  
exterior: 返回多边形的外环（LineString 对象）。  
interiors: 返回一个列表，包含多边形的所有内环（洞，LineString 对象）。  
Multi-Geometries (MultiPoint, MultiLineString, MultiPolygon):  
geoms: 返回一个列表，包含集合中的所有单个几何对象。  

In [48]:
import geopandas as gpd
from shapely.geometry import Point, LineString, Polygon, MultiPoint

# 创建一个 GeoDataFrame
data = {
    'name': ['City Hall', 'River', 'Park', 'Islands'],
    'geometry': [
        Point(-74.0060, 40.7128),
        LineString([(-74.05, 40.7), (-74.02, 40.75), (-73.98, 40.72)]),
        Polygon([(-74.01, 40.7), (-74.01, 40.71), (-74.00, 40.71), (-74.00, 40.7), (-74.01, 40.7)]),
        MultiPoint([Point(-73.9, 40.6), Point(-73.8, 40.5)])
    ]
}
gdf = gpd.GeoDataFrame(data, geometry='geometry', crs="EPSG:4326")
gdf.to_crs("EPSG:3857", inplace=True)  # 转换为 Web Mercator 投影

print("--- 几何属性示例 ---")
print(gdf)

# 访问 geom_type
print("\n几何类型:")
print(gdf.geometry.geom_type)

# 访问 bounds
print("\n边界框 (minx, miny, maxx, maxy):")
print(gdf.geometry.bounds)

# 访问 area (注意：对于地理坐标系，面积单位无意义，需先投影)
print("\n面积 (对于点和线为0.0):")
print(gdf.geometry.area)

# 访问 length (对于点和多边形为0.0)
print("\n长度 (对于点和多边形为0.0):")
print(gdf.geometry.length)

# 访问 centroid
print("\n质心:")
print(gdf.geometry.centroid)

# 访问 Point 的 x, y 坐标
print("\nPoint 的 X 坐标:")
print(gdf[gdf.geometry.geom_type == 'Point'].geometry.x)
print("\nPoint 的 Y 坐标:")
print(gdf[gdf.geometry.geom_type == 'Point'].geometry.y)

# 访问 Polygon 的 exterior
print("\nPolygon 的外环:")
print(gdf[gdf.geometry.geom_type == 'Polygon'].geometry.exterior)

# 访问 MultiPoint 的单个几何对象
print("\nMultiPoint 的组成部分:")
# 需要使用 apply 来访问每个 MultiPoint 对象的 geoms 属性
gdf['individual_geoms'] = gdf.geometry.apply(lambda geom: list(geom.geoms) if geom.geom_type.startswith('Multi') else None)
print(gdf['individual_geoms'])


--- 几何属性示例 ---
        name                                           geometry
0  City Hall                   POINT (-8238310.236 4970071.579)
1      River  LINESTRING (-8243208.293 4968191.93, -8239868....
2       Park  POLYGON ((-8238755.514 4968191.93, -8238755.51...
3    Islands  MULTIPOINT ((-8226510.37 4953519.587), (-82153...

几何类型:
0         Point
1    LineString
2       Polygon
3    MultiPoint
dtype: object

边界框 (minx, miny, maxx, maxy):
           minx          miny          maxx          maxy
0 -8.238310e+06  4.970072e+06 -8.238310e+06  4.970072e+06
1 -8.243208e+06  4.968192e+06 -8.235416e+06  4.975536e+06
2 -8.238756e+06  4.968192e+06 -8.237642e+06  4.969660e+06
3 -8.226510e+06  4.938869e+06 -8.215378e+06  4.953520e+06

面积 (对于点和线为0.0):
0    0.000000e+00
1    0.000000e+00
2    1.634665e+06
3    0.000000e+00
dtype: float64

长度 (对于点和多边形为0.0):
0        0.000000
1    14333.173150
2     5163.279563
3        0.000000
dtype: float64

质心:
0    POINT (-8238310.236 4970071.579)
1     